In [25]:
import json


import torch
import torch.nn as nn

import numpy as np
import torchvision
from torchvision import datasets, models, transforms

from torch.utils.data.sampler import Sampler


In [9]:
with open("./Feat_DenseNet201_VehicleID_test_medium.json", 'r') as f:
    data = json.load(f)

In [10]:
len(data)

1600

In [73]:
features_query, features_test = {}, {}
len_avg = 0
import random

for i, id in enumerate(data.keys()):
    #if i > 10:
    #    break
    if len(data[id]) > 1:
        #random.shuffle(data[id])
        features_query[id] = data[id][0]
        features_test[id] = data[id][1:]

In [74]:
device = torch.device("cuda:"+str(n_cuda) if torch.cuda.is_available() else "cpu")
cos= nn.CosineSimilarity(dim=1, eps=1e-6)

running_corrects = 0.0

running_data = 0.0
MATCH = {}
metric="MCD"

for cquery, fquery in features_query.items():

    fquery = torch.FloatTensor(fquery)
    fquery = fquery.unsqueeze(0)
    running_data += 1.0        

    listc =[]
    listd = []
    dmin = 1000000000
    class_found=""

    n_t=0
    n_ctest = len(features_test)
    for ctest, ftest in features_test.items():
        ftest = torch.FloatTensor(ftest)
        n_t += 1
        listc.append(ctest)
        l_d = []
        for f in ftest: 

            _dmin = 1000000000
            for g in fquery:
                if metric == "MCD":
                    d = 1 - cos(g.unsqueeze(0), f.unsqueeze(0))
                elif metric == "MED":
                    d = torch.norm(g - f)
                else:
                    raise Exception('metric should be either MED or MCD')
                _dmin = min(_dmin, d.item() )
                l_d.append(_dmin)
        l50_d = sorted(l_d)[:int(len(l_d)/2)+1]

        d = np.mean(l50_d)
        listd.append(d)
        if d < dmin:
            dmin = d
            class_found = ctest
    MATCH[cquery] = {}

    for i, (d, c) in enumerate(sorted(zip(listd, listc))):
        MATCH[cquery][i+1] = {"id": c, "distance":d}



KeyboardInterrupt: 

In [46]:
with open("MATCH_I2T_MCD_DenseNet201_VehicleID_test_medium","w") as tmpfile:
    tmpfile.write(json.dumps(MATCH))

In [72]:
rank1 = 0
rank5 = 0
n=0
for id, ranking in MATCH.items():
    #print(id, ranking[1]["id"])
    listrank5 = [ranking[i+1]["id"] for i in range(5)]
    if id==ranking[1]["id"]: rank1+=1
    if id in listrank5: rank5+=1
    n+=1
print(rank1, rank1/n)
print(rank5, rank5/n)

950 0.59375
1192 0.745
